# Для того, чтобы установить Invoke Ai, запустите код в каждой из форм.
Для запуска сервера потребуется аккаунт на [ngrok](https://dashboard.ngrok.com/login)

# To use Invoke Ai, launch the code in every form below.
You will need an account on [ngrok](https://dashboard.ngrok.com/login)

**[Repo](https://github.com/peaashmeter/invoke-ai-gui-colab)**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title Во время этого этапа Colab крашится, это нормально
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
#@title Подготовка репозитория Invoke Ai { display-mode: "form" }
%cd /home

!git clone -n https://github.com/invoke-ai/InvokeAI.git
%cd InvokeAI
!git checkout 69b15024a93ecfbbb555c956f745e9748c12f044

#dnspython fix
!echo '    - dnspython==2.2.1' >> environments-and-requirements/environment-lin-cuda.yml

!ln -sf environments-and-requirements/environment-lin-cuda.yml environment.yml
!ls -la

%cd ..

!git clone https://github.com/sphr2k/invoke-ai-gui-colab.git


%cd invoke-ai-gui-colab

%cp cross_attention_control.py ../InvokeAI/ldm/models/diffusion/cross_attention_control.py
%cp globals.py ../InvokeAI/ldm/invoke/globals.py

In [ ]:
#@title Установка зависимостей, в норме занимает ~10 минут { display-mode: "form" }
%cd ../InvokeAI
!pip install pyngrok --quiet
!conda env update
!source activate invokeai ; python scripts/configure_invokeai.py --skip-sd-weights --yes

%cp ../invoke-ai-gui-colab/models.yaml /root/invokeai/configs 



In [ ]:
#@title Установка модели Anything { display-mode: "form" }
import os

os.system('cd /')
os.system('mkdir -p /root/invokeai/models/ldm/stable-diffusion-v1')

%mkdir -p /root/invokeai/models/ldm/stable-diffusion-v1/
!ln -s /content/drive/MyDrive/models/Realistic_Vision_V1.4.ckpt /root/invokeai/models/ldm/stable-diffusion-v1/Realistic_Vision_V1.4.ckpt
!ls -l /root/invokeai/models/ldm/stable-diffusion-v1/Realistic_Vision_V1.4.ckpt

In [ ]:
#@title Запуск сервера { display-mode: "both" }
#@markdown Вставьте токен с [ngrok](https://dashboard.ngrok.com/get-started/your-authtoken) в поле перед выполнением кода.

ngrok_token = "" #@param {type:"string"}
nsfw_checker = 0 #@param {type:"slider", min:0, max:1, step:1}
used_vae = "Anything-V4.0.vae" #@param ["Anything-V4.0.vae", "vae-ft-mse-840000-ema-pruned"]

import os
from pyngrok import ngrok

ngrok.kill()
ngrok.set_auth_token(ngrok_token)
public_url = ngrok.connect(9090).public_url
print(f'Invoke Ai public url: {public_url}')

%cd /home/InvokeAI

model_name = "Anything_v4.5_vaefixed" #@param ["Anything_v4.5_vaefixed", "Anything_v4_vaefixed", "Anything_v3_vaefixed"]
if used_vae == "vae-ft-mse-840000-ema-pruned":
    if model_name == "Anything_v4.5_vaefixed":
      model_name = "Anything_v4.5_default"
    elif model_name == "Anything_v4_vaefixed":
      model_name = "Anything_v4_default"
    elif model_name == "Anything_v3_vaefixed":
      model_name = "Anything_v3_default"


if nsfw_checker:
    !source activate invokeai ; python scripts/invoke.py --web --model $model_name
else:
    !source activate invokeai ; python scripts/invoke.py --web --no-nsfw_checker --model $model_name

# После запуска сервера нужно перейти по первой ссылке из вывода (Invoke Ai public url)
Если интерфейс не прогрузился, надо перезагрузить страницу
---

